In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import math
import seaborn as sns
from cvx.covariance.combination import from_ewmas
from arch import arch_model
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from scipy.optimize import minimize
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

In [4]:
df=pd.read_csv(r'C:\Users\33640\OneDrive\Documents\GitHub\Covariance-Correlation-Volatility-Forecasting\SP500_daily_returns.csv')
nan_counts=df.isna().sum()
columns_to_keep = nan_counts[nan_counts <= 3].index
returns = df[columns_to_keep]
returns=returns.dropna()
returns.shape
dates=returns['Date'] 
returns=returns.drop('Date', axis=1)

In [5]:
def histo_cov(df, date, window):
    cov=np.zeros((len(df.columns),len(df.columns)))
    for i in range(date-window+1, date+1):
        cov+=np.outer(np.array(df.iloc[i,:]),np.array(df.iloc[i,:]))/(window)
    covs=pd.DataFrame(cov*250, columns=df.columns, index=df.columns)
    return(covs)

In [6]:
halflife_pairs=[(10, 21), (21, 63), (63, 125)]
def ewma_cov(df, date, halflife_pairs, window=500):    
    combinator = from_ewmas(df.iloc[max(0,date-1000):date,:],
                                halflife_pairs,
                                min_periods_vola=window,  # min periods for volatility estimation
                                min_periods_cov=window)  # min periods for correlation estimation

    # Solve combination problem and loop through combination results to get predictors
    covariance_predictors = {}
    for predictor in combinator.solve(window=10):  # lookback window for optimization
        # From predictor we can access predictor.time, predictor.mean (=0 here),
        # predictor.covariance, and predictor.weights
        covariance_predictors[predictor.time] = predictor.covariance
    return(covariance_predictors[date]*250)

In [7]:
# Step 1: Fit GARCH(1,1) to each asset's returns
def fit_garch(returns):
    # Rescaler les rendements
    returns_rescaled = returns * 100
    
    model = arch_model(returns_rescaled, vol='Garch', p=1, q=1)
    garch_fit = model.fit(disp="off")
    
    # Revenir à l'échelle d'origine pour les résidus
    residuals = garch_fit.resid / 100
    volatilities = garch_fit.conditional_volatility / 100
    return volatilities, residuals
# Step 2: DCC-GARCH log-likelihood function
def dcc_garch_log_likelihood(params, residuals):
    """
    Calcule la log-vraisemblance du modèle DCC-GARCH.
    
    Paramètres:
    params (tuple): Paramètres alpha_2 et beta_2.
    residuals (np.array): Matrice des résidus standardisés (T x N).
    
    Retourne:
    float: La valeur de la log-vraisemblance négative.
    """
    alpha_2, beta_2 = params
    n_assets = residuals.shape[1]
    
    # Initialisation de Q_bar et Q_t
    T = residuals.shape[0]
    Q_bar = np.cov(residuals.T)  # Matrice de corrélation inconditionnelle
    Q_t = Q_bar.copy()
    
    log_likelihood = 0
    epsilon = 1e-5  # Petit terme de régularisation pour éviter les matrices singulières
    
    for t in range(T):
        # Mise à jour de Q_t selon la formule DCC
        Q_t = (1 - alpha_2 - beta_2) * Q_bar + alpha_2 * np.outer(residuals[t], residuals[t]) + beta_2 * Q_t
        
        # Régularisation de la diagonale de Q_t pour éviter les valeurs négatives
        diag_Q_t = np.diag(Q_t)
        diag_Q_t = np.clip(diag_Q_t, epsilon, None)  # Clip des valeurs négatives ou proches de zéro
        Q_t = np.diag(diag_Q_t) + (Q_t - np.diag(np.diag(Q_t)))  # Remplacer la diagonale
        
        # Ajouter un petit terme de régularisation à Q_t
        Q_t += epsilon * np.eye(n_assets)
        
        # Normaliser Q_t pour obtenir R_t (matrice de corrélation dynamique)
        D_t = np.diag(1 / np.sqrt(np.diag(Q_t)))
        R_t = D_t @ Q_t @ D_t
        
        # Vérifier les valeurs propres de R_t et corriger si nécessaire
        eigenvalues = np.linalg.eigvals(R_t)
        if np.any(eigenvalues <= 0):
            R_t += epsilon * np.eye(n_assets)
        
        # Calcul de la log-vraisemblance pour chaque étape t
        try:
            term1 = np.log(np.linalg.det(R_t))  # Log du déterminant de R_t
        except np.linalg.LinAlgError:
            # En cas de matrice singulière
            return np.inf  # Retourner une grande valeur pour pénaliser l'optimisation
        
        term2 = residuals[t].T @ np.linalg.pinv(R_t) @ residuals[t]  # epsilon_t' R_t^{-1} epsilon_t
        term3 = residuals[t].T @ residuals[t]  # epsilon_t' epsilon_t
        log_likelihood += term1 + term2 + term3  # Ajouter les trois termes pour chaque t
    
    # Retourner la log-vraisemblance négative
    return 0.5 * log_likelihood

# Step 6: Optimization of alpha_2 and beta_2
def optimize_dcc_garch(returns):
    """
    Optimise les paramètres alpha_2 et beta_2 du modèle DCC-GARCH en utilisant une fenêtre de données.
    
    Paramètres:
    returns (pd.DataFrame): DataFrame des rendements journaliers des actifs (chaque colonne représente un actif).
    window (str): Fenêtre temporelle pour la sélection des données ('6M' pour 6 mois, '1Y' pour 1 an, etc.).
    
    Retourne:
    tuple: Paramètres optimisés alpha_2 et beta_2.
    """
    n_assets = returns.shape[1]
    
    # Ajuster GARCH(1,1) sur chaque actif et standardiser les résidus
    volatilities = np.zeros_like(returns)
    residuals = np.zeros_like(returns)
    
    for i in range(n_assets):
        volatilities[:, i], residuals[:, i] = fit_garch(returns.iloc[:, i])
    
    standardized_residuals = residuals / volatilities
    
    # Valeurs initiales pour alpha_2 et beta_2
    initial_params = np.array([0.05, 0.9])
    
    # Contraintes sur les paramètres
    bounds = [(0, 1), (0, 0.95)]  # Limiter beta_2 à 0.95 pour éviter une convergence lente
    
    # Minimiser la log-vraisemblance négative
    result = minimize(dcc_garch_log_likelihood, initial_params, args=(standardized_residuals), bounds=bounds, method='SLSQP', options={'disp': True})
    
    if result.success:
        optimized_params = result.x
        return optimized_params
    else:
        raise ValueError("Optimization failed")

def predict_covariance_dcc_garch(df, date, window, horizon=1):
    """
    Calcule la matrice de covariance prédite par le modèle DCC-GARCH à un horizon donné.
    
    Paramètres:
    returns (pd.DataFrame): DataFrame des rendements journaliers des actifs (chaque colonne représente un actif).
    window (str): Fenêtre temporelle pour la sélection des données ('6M' pour 6 mois, '1Y' pour 1 an, etc.).
    horizon (int): Nombre de jours dans le futur pour lesquels la covariance doit être prédite.
    
    Retourne:
    pd.DataFrame: Matrice de covariance prédite à l'horizon spécifié.
    """
    returns=df.iloc[date-window:date,:]
    n_assets = returns.shape[1]
    
    # Ajuster GARCH(1,1) sur chaque actif et standardiser les résidus
    volatilities = np.zeros_like(returns)
    residuals = np.zeros_like(returns)
    
    for i in range(n_assets):
        volatilities[:, i], residuals[:, i] = fit_garch(returns.iloc[:, i])
    
    standardized_residuals = residuals / volatilities
    
    # Initialiser les paramètres DCC-GARCH avec les valeurs optimisées
    optimized_alpha2, optimized_beta2 = optimize_dcc_garch(returns)
    
    # Initialisation de Q_bar et Q_t
    Q_bar = np.cov(standardized_residuals.T)
    Q_t = Q_bar.copy()
    T = standardized_residuals.shape[0]
    
    # Utiliser les paramètres optimisés pour prédire la covariance
    for t in range(T):
        Q_t = (1 - optimized_alpha2 - optimized_beta2) * Q_bar + optimized_alpha2 * np.outer(standardized_residuals[t], standardized_residuals[t]) + optimized_beta2 * Q_t
    
    # Normaliser Q_t pour obtenir la matrice de corrélation dynamique R_t à la dernière date
    D_t = np.diag(1 / np.sqrt(np.diag(Q_t)))
    R_t = D_t @ Q_t @ D_t
    
    # Prédire la matrice de covariance à horizon k jours
    last_volatilities = volatilities[-1, :]
    D_last = np.diag(last_volatilities)
    predicted_covariance = D_last @ R_t @ D_last
    
    # Boucle pour étendre la prédiction à horizon k jours
    for _ in range(horizon):
        # Mettre à jour Q_t pour chaque jour à venir
        Q_t = (1 - optimized_alpha2 - optimized_beta2) * Q_bar + optimized_beta2 * Q_t
        
        # Normaliser Q_t pour obtenir R_t (corrélation dynamique après k jours)
        D_t = np.diag(1 / np.sqrt(np.diag(Q_t)))
        R_t = D_t @ Q_t @ D_t
        
        # Mise à jour des volatilités pour chaque jour
        predicted_covariance = D_last @ R_t @ D_last
    
    # Retourner la matrice de covariance sous forme de DataFrame avec les colonnes et index des actifs
    return pd.DataFrame(predicted_covariance*250, index=returns.columns, columns=returns.columns)

In [8]:
def reg_RW_vols(df, date, windows):
    Har_vols = []
    for actif in df.columns:
        vols=[[] for i in range(len(windows))]
        vol_realized = []
        M=max(windows)
        for t in range(date-250, date ):
            for i, window in enumerate(windows):
                if window>1:
                    vols[i].append(df.loc[t - window+1:t, actif].std() * np.sqrt(250))
                else:
                    vols[i].append(np.sqrt(df.loc[t , actif]**2 * 250))
            vol_realized.append(np.sqrt(df.loc[t+1 , actif]**2 * 250))
        vol_data = pd.DataFrame({f'vol_{i+1}': vols[i] for i in range(len(windows))})
        vol_data['vol_realized'] = vol_realized
        vol_data = vol_data.dropna()
        
        X = vol_data[[f'vol_{i+1}' for i in range(len(windows))]]
        X = add_constant(X)
        y = vol_data['vol_realized']
        model = OLS(y, X).fit()
        latest_vols={}
        latest_vols['intercept'] = [1]
        for i, window in enumerate(windows):
            if window>1:
                latest_vols[f'vol_{i+1}']=[df.loc[date - window+1:date, actif].std() * np.sqrt(250)] 
            else:
                latest_vols[f'vol_{i+1}']=[np.sqrt(df.loc[date, actif]**2 * np.sqrt(250))] 
        
        latest_data = pd.DataFrame(latest_vols)
        vol_forecast = model.predict(latest_data)
        Har_vols.append(vol_forecast.iloc[0])
    
    return (np.array(Har_vols))



def ewma_covariance(df_returns, date, beta=0.94):
    # Nombre d'actifs (colonnes)
    n_assets = df_returns.shape[1]
    
    # Calcul de la moyenne des rendements
    mean_returns = df_returns.mean()
    
    # Initialisation de la matrice de covariance EWMA
    ewma_cov = np.zeros((n_assets, n_assets))
    
    # Matrice de covariance initiale basée sur les premières données
    for t in range(date):
        # Déviation des rendements par rapport à la moyenne
        diff = (df_returns.iloc[t, :] - mean_returns).values.reshape(-1, 1)
        
        # Limiter la magnitude des valeurs pour éviter l'overflow
        if np.any(np.abs(diff) > 1e5):
            diff = np.clip(diff, -1e5, 1e5)
        
        # Mise à jour de la matrice de covariance EWMA
        ewma_cov = (1 - beta) * np.dot(diff, diff.T) + beta * ewma_cov
    
    # Transformation en DataFrame pour un affichage plus lisible
    ewma_cov_df = pd.DataFrame(ewma_cov, index=df_returns.columns, columns=df_returns.columns)
    
    return ewma_cov_df

def ewma_correlation(df_returns, date, beta=0.97):
    # Calcul de la matrice de covariance EWMA
    ewma_cov_df = ewma_covariance(df_returns, date, beta)
    
    # Calcul des écarts-types EWMA
    ewma_std = np.sqrt(np.diag(ewma_cov_df))
    
    # Matrice de corrélation EWMA
    ewma_corr = ewma_cov_df / np.outer(ewma_std, ewma_std)
    
    # Transformation en DataFrame
    ewma_corr_df = pd.DataFrame(ewma_corr, index=df_returns.columns, columns=df_returns.columns)
    
    return ewma_corr_df

def covariance_matrix(volatility_vector, correlation_matrix):
    # volatility_vector: 1D numpy array or list of volatilities (standard deviations)
    # correlation_matrix: 2D numpy array or DataFrame of correlations
    
    # Convert the volatility vector into a diagonal matrix
    vol_diag = np.diag(volatility_vector)
    
    # Calculate the covariance matrix: Cov = D * Corr * D
    cov_matrix = vol_diag @ correlation_matrix @ vol_diag
    
    return cov_matrix

In [9]:
def sigmahat(Y,k=None):
    None, np.nan or int
    #Pre-Conditions: Y is a valid pd.dataframe and optional arg- k which can be
    #
    #Post-Condition: Sigmahat dataframe is returned
    #Set df dimensions
    N = Y.shape[0]
    p = Y.shape[1]
    #default setting
    if (k is None or math.isnan(k)):
        Y = Y.sub(Y.mean(axis=0), axis=1)
        k = 1
    #vars
    n = N-k
    c = p/n
    sample = pd.DataFrame(np.matmul(Y.T.to_numpy(),Y.to_numpy()))/n
    sample = (sample+sample.T)/2
    #make symmetrical
    #Spectral decomp
    lambda1, u = np.linalg.eigh(sample)
    lambda1 = lambda1.real.clip(min=0)
    dfu = pd.DataFrame(u,columns=lambda1) #create df with column names lambda
    dfu.sort_index(axis=1,inplace = True)
    lambda1 = dfu.columns
    h = (min(c**2,1/c**2)**0.35)/p**0.35
    #smoothing parameter
    invlambda = 1/lambda1[max(1,p-n+1)-1:p] #inverse of (non-null) eigenvalues
    dfl = pd.DataFrame()
    dfl['lambda'] = invlambda
    Lj = dfl[np.repeat(dfl.columns.values,min(p,n))]
    Lj = pd.DataFrame(Lj.to_numpy())
    Lj_i = Lj.subtract(Lj.T)
    #like 1/lambda_j
    #Reset column names
    #like (1/lambda_j)-(1/lambda_i)
    theta = Lj.multiply(Lj_i).div(Lj_i.multiply(Lj_i).add(Lj.multiply(Lj)*h**2)).mean(axis = 0)
    #smoothed Stein shrinker
    Htheta = Lj.multiply(Lj*h).div(Lj_i.multiply(Lj_i).add(Lj.multiply(Lj)*h**2)).mean(axis = 0)
    Atheta2 = theta**2+Htheta**2
    if p<=n:
        delta = 1 / ((1-c)**2*invlambda+2*c*(1-c)*invlambda*theta \
        +c**2*invlambda*Atheta2)
        delta = delta.to_numpy()
    else:
        delta0 = 1/((c-1)*np.mean(invlambda.to_numpy()))
        delta = np.repeat(delta0,p-n)
        delta = np.concatenate((delta, 1/(invlambda*Atheta2)), axis=None)
    deltaQIS = delta*(sum(lambda1)/sum(delta))
    temp1 = dfu.to_numpy()
    temp2 = np.diag(deltaQIS)
    temp3 = dfu.T.to_numpy().conjugate()
    #reconstruct covariance matrix
    #preserve trace
    sigmahat = pd.DataFrame(np.matmul(np.matmul(temp1,temp2),temp3), index=Y.columns, columns=Y.columns)
    return sigmahat

In [65]:
def minimum_variance_portfolio(cov_matrix):
    """
    Calcule les pondérations du portefeuille de variance minimale
    pour une matrice de covariance donnée. Si la matrice est mal conditionnée
    ou non inversible, minimise directement la variance via une optimisation
    avec une contrainte supplémentaire : aucun poids ne dépasse 0.10.
    """
    n = len(cov_matrix)
    ones = np.ones(n)

    # Tenter l'inversion de la matrice de covariance
    try:
        inv_cov_matrix = np.linalg.inv(cov_matrix)
        weights = inv_cov_matrix @ ones / (ones.T @ inv_cov_matrix @ ones)
        
        # Appliquer une contrainte post-inversion (pour les poids ≤ 0.10)
        weights = np.clip(weights, 0, 0.10)
        weights /= np.sum(weights)  # Re-normaliser pour que la somme soit égale à 1
    except np.linalg.LinAlgError:
        # Minimisation numérique si la matrice n'est pas inversible
        def portfolio_variance(weights):
            return weights.T @ cov_matrix @ weights

        # Contrainte : Somme des poids = 1
        constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

        # Les poids doivent être entre 0 et 0.10
        bounds = [(0, 0.10) for _ in range(n)]

        # Résolution du problème d'optimisation
        result = minimize(portfolio_variance, 
                          x0=ones/n, 
                          bounds=bounds, 
                          constraints=constraints, 
                          method='SLSQP')

        # Extraire les poids optimaux
        if result.success:
            weights = result.x
        else:
            raise ValueError("L'optimisation a échoué.")

    return weights

def ERC_portfolio(forecasting):
    cov=forecasting
    n=len(cov.columns)
    def objective(weights):
        return(1/2*weights.T@cov@weights-1/n*np.sum(np.array([np.log(weights[i]) for i in range(n)])))
    initial_weights=np.array([1/n+2]*n)
    bounds=tuple((1,10) for _ in range(n))
    result=minimize(objective, initial_weights,method='SLSQP', bounds=bounds,constraints=None)
    weight=result.x/np.sum(result.x)
    return(weight)

def max_div_portfolio(forecasting):
    cov=forecasting
    n=len(cov.columns)
    def objective(weights):
        return(weights.T@cov@weights)
    initial_weights=np.array([1/n]*n)
    bounds=tuple((0,100) for _ in range(n))
    constraints=({'type':'eq','fun': lambda weights:np.dot(weights, np.sqrt(np.diag(cov)))-1})
    result=minimize(objective, initial_weights,method='SLSQP', bounds=bounds,constraints=constraints)
    weight=result.x/np.sum(result.x)
    return(weight)

def portfolio_standard_deviation(returns, cov_matrices):
    """
    Construit les portefeuilles de variance minimale à partir d'une liste
    de matrices de covariance et calcule l'écart-type (standard deviation)
    de la stratégie.
    
    :param returns: DataFrame avec les rendements journaliers par actifs.
    :param cov_matrices: Liste de matrices de covariance.
    :return: Écart-type de la stratégie de portefeuilles de variance minimale.
    """
    n = len(returns)  # Nombre total de jours dans les rendements
    L = len(cov_matrices)  # Nombre de matrices de covariance
    X = n // L  # Nombre de jours pendant lesquels chaque portefeuille est maintenu
    
    portfolio_returns = []  # Liste pour stocker les rendements des portefeuilles

    # Boucle sur chaque matrice de covariance pour créer un portefeuille
    for i, cov_matrix in enumerate(cov_matrices):
        # Calculer les pondérations du portefeuille de variance minimale
        weights = minimum_variance_portfolio(cov_matrix)
        
        # Calculer les rendements du portefeuille pour les X jours correspondants
        start = i * X
        end = (i + 1) * X if (i + 1) * X <= n else n  # S'assurer de ne pas dépasser n
        returns_subset = returns.iloc[start:end, :]  # Sous-ensemble des rendements
        portfolio_return = returns_subset @ weights  # Rendement du portefeuille sur cette période
        
        # Ajouter les rendements du portefeuille à la liste
        portfolio_returns.extend(portfolio_return)
    
    # Convertir en série pandas
    portfolio_returns = pd.Series(portfolio_returns)
    
    # Calcul de l'écart-type de la stratégie de portefeuilles
    strategy_std = portfolio_returns.std()
    
    return strategy_std*np.sqrt(252)

def EWportfolio_standard_deviation(returns, cov_matrices, risk_free_rate=0.0):
    """
    Construit les portefeuilles de variance minimale à partir d'une liste
    de matrices de covariance et calcule l'écart-type (standard deviation)
    de la stratégie.
    
    :param returns: DataFrame avec les rendements journaliers par actifs.
    :param cov_matrices: Liste de matrices de covariance.
    :return: Écart-type de la stratégie de portefeuilles de variance minimale.
    """
    n = len(returns)  # Nombre total de jours dans les rendements
    L = len(cov_matrices)  # Nombre de matrices de covariance
    X = n // L  # Nombre de jours pendant lesquels chaque portefeuille est maintenu
    print(X)
    print(n)
    portfolio_returns = []  # Liste pour stocker les rendements des portefeuilles

    # Boucle sur chaque matrice de covariance pour créer un portefeuille
    for i, cov_matrix in enumerate(cov_matrices):
        # Calculer les pondérations du portefeuille de variance minimale
        weights = np.ones(returns.shape[1]) / returns.shape[1]
        
        # Calculer les rendements du portefeuille pour les X jours correspondants
        start = i * X
        end = (i + 1) * X if (i + 1) * X <= n else n  # S'assurer de ne pas dépasser n
        returns_subset = returns.iloc[start:end, :]  # Sous-ensemble des rendements
        portfolio_return = returns_subset @ weights  # Rendement du portefeuille sur cette période
        
        # Ajouter les rendements du portefeuille à la liste
        portfolio_returns.append((1 + portfolio_return).prod() - 1)

    # Convertir en série pandas
    portfolio_returns = pd.Series(portfolio_returns)

    # Calcul de l'écart-type de la stratégie de portefeuilles (volatilité annualisée)
    strategy_std = portfolio_returns.std() * np.sqrt(50)

    # Calcul du rendement cumulé
    cumulative_return = (1 + portfolio_returns).prod() - 1

    # Calcul du rendement annualisé cumulé
    annualized_return = (1 + cumulative_return) ** (50 / L) - 1

    # Calcul du Sharpe ratio
    excess_returns = portfolio_returns - (risk_free_rate / 50)
    sharpe_ratio = (excess_returns.mean() * 50) / (strategy_std )

    # Calcul du maximum drawdown
    cumulative_returns = (1 + portfolio_returns).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()

    return {
        'Annualized Standard Deviation': strategy_std,
        'Max Drawdown': max_drawdown,
        'Annualized Return': annualized_return,
        'annualized mean return':excess_returns.mean() * 50,
        'Sharpe Ratio': sharpe_ratio
    }

def portfolio_standard_deviation(returns, cov_matrices, risk_free_rate=0.0):
    """
    Construit les portefeuilles de variance minimale à partir d'une liste
    de matrices de covariance et calcule l'écart-type (standard deviation),
    le maximum drawdown, le rendement annualisé cumulé, le Sharpe ratio,
    la distance ERC et la diversification.

    :param returns: DataFrame avec les rendements journaliers par actifs.
    :param cov_matrices: Liste de matrices de covariance.
    :param risk_free_rate: Taux sans risque pour calculer le Sharpe ratio.
    :return: Un dictionnaire avec l'écart-type annualisé, le max drawdown, 
             le rendement annualisé cumulé, le Sharpe ratio, la distance ERC,
             et la diversification.
    """
    n = len(returns)  # Nombre total de jours dans les rendements
    L = len(cov_matrices)  # Nombre de matrices de covariance
    X = n // L  # Nombre de jours pendant lesquels chaque portefeuille est maintenu

    portfolio_returns = []  # Liste pour stocker les rendements des portefeuilles
    erc_distances = []  # Liste pour stocker les distances ERC
    diversifications = []  # Liste pour stocker la diversification

    # Boucle sur chaque matrice de covariance pour créer un portefeuille
    for i, cov_matrix in enumerate(cov_matrices):
        # Calculer les pondérations du portefeuille ERC
        weights =ERC_portfolio(cov_matrix)

        # Calculer les rendements du portefeuille pour les X jours correspondants
        start = i * X
        end = (i + 1) * X if (i + 1) * X <= n else n  # S'assurer de ne pas dépasser n
        returns_subset = returns.iloc[start:end, :]  # Sous-ensemble des rendements
        portfolio_return = returns_subset @ weights  # Rendement du portefeuille sur cette période

        # Ajouter les rendements du portefeuille à la liste
        portfolio_returns.append((1 + portfolio_return).prod() - 1)

        # Calculer la volatilité totale du portefeuille
        portfolio_volatility = portfolio_return.std()*np.sqrt(250)

        # Contribution marginale au risque de chaque actif
        marginal_risk_contributions = returns_subset.cov() @ weights / portfolio_volatility

        # Contribution totale au risque de chaque actif
        total_risk_contributions = weights * marginal_risk_contributions

        # Moyenne des contributions totales au risque
        mean_trc = np.mean(total_risk_contributions)

        # Calcul de la distance ERC : somme des écarts absolus
        erc_distance = np.sum(np.abs(total_risk_contributions - mean_trc))
        erc_distances.append(erc_distance)

        # Calcul de la diversification
        asset_volatilities = np.sqrt(np.diag(returns_subset.cov()))  # Volatilité de chaque actif
        diversification = (weights.T @ asset_volatilities) / portfolio_volatility
        diversifications.append(diversification)

    # Convertir en série pandas
    portfolio_returns = pd.Series(portfolio_returns)

    # Calcul de l'écart-type de la stratégie de portefeuilles (volatilité annualisée)
    strategy_std = portfolio_returns.std() * np.sqrt(50)

    # Calcul du rendement cumulé
    cumulative_return = (1 + portfolio_returns).prod() - 1

    # Calcul du rendement annualisé cumulé
    annualized_return = (1 + cumulative_return) ** (50 / L) - 1

    # Calcul du Sharpe ratio
    excess_returns = portfolio_returns - (risk_free_rate / 50)
    sharpe_ratio = (excess_returns.mean() * 50) / (strategy_std)

    # Calcul du maximum drawdown
    cumulative_returns = (1 + portfolio_returns).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()

    # Moyenne des distances ERC sur toutes les périodes
    avg_erc_distance = np.mean(erc_distances)

    # Moyenne de la diversification sur toutes les périodes
    avg_diversification = np.mean(diversifications)

    return {
        'Annualized Standard Deviation': strategy_std,
        'Max Drawdown': max_drawdown,
        'Annualized Return': annualized_return,
        'Annualized Mean Return': excess_returns.mean() * 50,
        'Sharpe Ratio': sharpe_ratio,
        'ERC Distance': avg_erc_distance,  # Ajout de la distance ERC
        'Diversification': avg_diversification  # Ajout de la diversification
    }



In [66]:
portfolio_standard_deviation(returns.iloc[4966:returns.shape[0]-1,:], Histo_cov21)

{'Annualized Standard Deviation': 0.1901686043209643,
 'Max Drawdown': -0.3708497799076201,
 'Annualized Return': 0.09388400323505408,
 'Annualized Mean Return': 0.10797615330565798,
 'Sharpe Ratio': 0.5677916903855335,
 'ERC Distance': 0.0005195253762846874,
 'Diversification': 0.1533095005643699}

In [67]:
portfolio_standard_deviation(returns.iloc[4966:returns.shape[0]-1,:], Histo_cov125)

{'Annualized Standard Deviation': 0.20157466339036928,
 'Max Drawdown': -0.3723194765890322,
 'Annualized Return': 0.09220502384315044,
 'Annualized Mean Return': 0.10855332407817102,
 'Sharpe Ratio': 0.5385266295494031,
 'ERC Distance': 0.0004946713400519605,
 'Diversification': 0.14183499708880315}

In [68]:
portfolio_standard_deviation(returns.iloc[4966:returns.shape[0]-1,:], Ewma_cov)

{'Annualized Standard Deviation': 0.20060237484166465,
 'Max Drawdown': -0.37231947658904063,
 'Annualized Return': 0.09252479683249137,
 'Annualized Mean Return': 0.10864888573028895,
 'Sharpe Ratio': 0.5416131579501263,
 'ERC Distance': 0.0004958530687788928,
 'Diversification': 0.142970476482191}

In [69]:
portfolio_standard_deviation(returns.iloc[4966:returns.shape[0]-1,:], reg_cov)

{'Annualized Standard Deviation': 0.20086637651533054,
 'Max Drawdown': -0.37051216472831355,
 'Annualized Return': 0.09226710201851751,
 'Annualized Mean Return': 0.10845769650698622,
 'Sharpe Ratio': 0.5399494847695852,
 'ERC Distance': 0.0004956245966942916,
 'Diversification': 0.14248742019925292}

In [ ]:
portfolio_standard_deviation(returns.iloc[4966:returns.shape[0]-1,:], NL21_cov)

In [47]:
portfolio_standard_deviation(returns.iloc[4966:returns.shape[0]-1,:], NL125_cov)

{'Annualized Standard Deviation': 0.17007556964843656,
 'Max Drawdown': -0.3139740601221061,
 'Annualized Return': 0.10708473122507378,
 'Annualized Mean Return': 0.11625809505634006,
 'Sharpe Ratio': 0.6835672830416345,
 'ERC Distance': 0.000837310502840542,
 'Diversification': 0.15275411484927634}

In [62]:
portfolio_standard_deviation(returns.iloc[4966:returns.shape[0]-1,:], NL125_cov)

KeyboardInterrupt: 

In [37]:
Histo_cov21=[]
Histo_cov125=[]
Ewma_cov=[]
dccgarch_cov=[]
reg_cov=[]
NL21_cov=[]
NL125_cov=[]
halflife_pairs=[(10, 21), (21, 63), (63, 125)]
for t in range(4966, returns.shape[0]-1, 5):
    if t%50==0:
        print(t)
    i=0
    Histo_cov21.append(histo_cov(returns, t, 21))
    Histo_cov125.append(histo_cov(returns, t, 125))
    Ewma_cov.append(ewma_cov(returns.iloc[:,:50], t, halflife_pairs, 500))
    reg_cov.append(covariance_matrix(reg_RW_vols(returns, t, [5,21]), ewma_correlation(returns, t, 0.97)))
    NL21_cov.append(sigmahat(returns.iloc[t-21:t,:],k=None))
    NL125_cov.append(sigmahat(returns.iloc[t-125:t,:],k=None))

In [51]:
def ewma_cov(df, date, window, beta):
    cov=np.zeros((len(df.columns),len(df.columns)))
    for i in range(date-window+1, date+1):
        cov=(beta)*cov+(1-beta)*np.outer(np.array(df.iloc[i,:]),np.array(df.iloc[i,:]))
    covs=pd.DataFrame(cov*250, columns=df.columns, index=df.columns)
    return(covs)

In [54]:
Ewma_cov=[]
for t in range(4966, returns.shape[0]-1, 5):
    Ewma_cov.append(ewma_cov(returns, t, 500, 0.96))